In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
#set directory for importinh my_tools,py
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import plot_dic, file_dic
from datetime import datetime
import glob
%matplotlib qt

In [2]:
#set path. Change accoridngly
path_file = '/media/onno/Algemeen/Thesis/'
path = '/media/onno/Algemeen/Thesis/GFS_mean_season/'

Mean variable error for each season

In [3]:
#Categorize each month for season
variablez = ['envelope']
seasons = {'MAM':[3,4,5],
          'JJA':[6,7,8],
          'SON':[9,10,11],
          'DJF':[12,1,2]}
seasons = {'MMA':[3,4,5]}
#Loop through season
for variable in variablez:
    print(variable)
    #get file names from imported dictionary
    file_gfs = file_dic[variable]['GFS']
    file_era = file_dic[variable]['ERA5']
    #open netcdf files as xarray datasets
    gfs = xr.open_dataset(path_file+file_gfs,decode_times=False).squeeze()
    era = xr.open_dataset(path_file+file_era,decode_times=False).squeeze()
    #change dimension names of ERA5 dataset
    if list(era.coords)[1]=='longitude':
        era=era.rename({'longitude':'lon'})
    if list(era.coords)[2]=='latitude':
        era=era.rename({'latitude':'lat'})

    #Convert dates to standard datetime values
    gfs['time']=my_tools.convert_date_gefs_r(gfs.time.values)
    era['time']=my_tools.convert_date_era_r(era.time.values)

    #Select only data for 24h intervals
    gfs = gfs.sel(lead=gfs.lead.values[[bool(i) for i in gfs.lead%24==0]])    
    for key in seasons:
        #loop through lead days
        for i in range(11):
            #Select days for which GEFS data is present and make datasets equally long
            gfs_d = gfs.sel(time=[i for i in pd.date_range('1984-12-1','2019-11-{}'.format(30-i))],lead=i*24)
            era_d = era.sel(time=[i for i in pd.date_range('1984-12-{}'.format(1+i),'2019-11-30')])        
            #Make boolean array for each day that is present in season        
            idxz = np.isin(gfs_d.time.dt.month,seasons[key])
            gfs_season = gfs_d[plot_dic[variable]['data_variable']][idxz] ; era_season = era_d[plot_dic[variable]['data_variable']][idxz]
            #Check if dates are correct
            assert(np.all(np.isin([(j-pd.Timedelta(i,'d')).month for j in era_season.time.values],seasons[key])))
            assert(np.all(np.isin([pd.to_datetime(j).month for j in gfs_season.time.values],seasons[key])))
            gfs_season_mean = gfs_season.mean(dim='time')
            era_season_mean = era_season.mean(dim='time')
            diff = gfs_season_mean - era_season_mean
            diff.to_netcdf(path+'/GFS_mean_error_{}_{}_day_{:02d}_globe_non_mars.nc'.format(plot_dic[variable]['filename'],key,i))



envelope
